# SQL-проект Исследование базы данных StackOverflow

**Данные:** 
База данных StackOverflow — сервиса вопросов и ответов о программировании. StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.

**Задача:**
Проанализировать данные, произвести выгрузки и ответить на поставленные вопросы с помощью SQL.

## Задания

### Задание 1

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».<br>
'''<br>
SELECT COUNT(p.id)<br>
FROM stackoverflow.post_types AS pt<br>
JOIN stackoverflow.posts AS p ON pt.id = p.post_type_id<br>
WHERE pt.type = 'Question'<br>
 AND (p.favorites_count >= 100 OR p.score > 300)<br>
'''

### Задание 2

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.<br>
'''<br>
WITH pre_table AS (<br>
SELECT<br>
  CAST(creation_date AS date),<br>
  COUNT(p.id) AS uid_cnt<br>
FROM stackoverflow.posts AS p<br>
JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id<br>
WHERE pt.type = 'Question' <br>
  AND p.creation_date BETWEEN '2008-11-01' AND '2008-11-19'<br>
GROUP BY CAST(creation_date AS date))<br>
SELECT ROUND(AVG(uid_cnt))<br>
FROM pre_table<br>
'''

### Задание 3

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.<br>
'''<br>
SELECT COUNT (DISTINCT u.id)<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.badges AS b ON b.user_id = u.id<br>
WHERE CAST(b.creation_date AS date) = CAST(u.creation_date AS date)<br>
'''

### Задание 4

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?<br>
'''<br>
SELECT COUNT(DISTINCT p.id)<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.posts AS p ON p.user_id = u.id<br>
JOIN stackoverflow.votes AS v ON v.post_id = p.id<br>
WHERE u.display_name = 'Joel Coehoorn'<br>
'''

### Задание 5

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.<br>
'''<br>
SELECT *,<br>
       RANK() OVER(ORDER BY id DESC)<br>
FROM stackoverflow.vote_types<br>
ORDER BY id<br>
'''

### Задание 6

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.<br>
'''<br>
SELECT<br>
  u.id,<br>
  COUNT(v.id) AS voice_cnt<br>
FROM stackoverflow.vote_types AS vt<br>
JOIN stackoverflow.votes AS v ON v.vote_type_id = vt.id<br>
JOIN stackoverflow.users AS u ON u.id = v.user_id<br>
WHERE vt.name = 'Close'<br>
GROUP BY u.id<br>
ORDER BY voice_cnt DESC, u.id DESC<br>
LIMIT 10;<br>
'''

### Задание 7

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:

- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.<br>
'''<br>
WITH pre_table AS (<br>
SELECT<br>
  u.id AS uid,<br>
  COUNT(b.id) AS bad_cnt<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.badges AS b ON b.user_id = u.id<br>
WHERE b.creation_date BETWEEN '2008-11-15' AND '2008-12-16'<br>
GROUP BY uid)<br>
SELECT<br>
  *,<br>
  DENSE_RANK() OVER(ORDER BY bad_cnt DESC) AS rang<br>
FROM pre_table<br>
ORDER BY bad_cnt DESC, uid<br>
LIMIT 10<br>
'''

### Задание 8
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:

- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.<br>
'''<br>
SELECT<br>
  p.title AS head,<br>
  u.id AS uid,<br>
  p.score,<br>
  ROUND(AVG(score) OVER(PARTITION BY u.id))<br>
FROM stackoverflow.posts AS p<br>
JOIN stackoverflow.users AS u ON u.id = p.user_id<br>
WHERE p.score != 0 AND p.title IS NOT NULL<br>
GROUP BY uid, head, score<br>
'''

### Задание 9
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.<br>
'''<br>
WITH pre_users AS(<br>
SELECT u.id, COUNT(b.id) AS cnt<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.badges AS b ON b.user_id = u.id<br>
GROUP BY u.id)<br>
SELECT DISTINCT(p.title)<br>
FROM pre_users AS pre<br>
JOIN stackoverflow.posts AS p ON p.user_id = pre.id<br>
WHERE pre.cnt > 1000 AND p.title IS NOT NULL<br>
'''

### Задание 10
Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.<br>
'''<br>
SELECT <br>
  u.id,<br>
  u.views,<br>
  CASE<br>
    WHEN u.views >= 350 THEN 1<br>
    WHEN u.views >= 100 THEN 2<br>
    WHEN u.views < 100 THEN 3<br>
  END<br>
FROM stackoverflow.users AS u<br>
WHERE u.location LIKE '%United States%' AND u.views > 0<br>
'''

### Задание 11
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.<br>
'''<br>
WITH max_views_users AS(<br>
WITH pre_users AS (<br>
SELECT<br>
  u.id,<br>
  u.views,<br>
  CASE<br>
    WHEN u.views >= 350 THEN 1<br>
    WHEN u.views >= 100 THEN 2<br>
    ELSE 3<br>
  END AS rang<br>
FROM stackoverflow.users AS u<br>
WHERE u.location LIKE '%United States%' AND u.views > 0)<br>
SELECT <br>
  id,<br>
  rang,<br>
  views,<br>
  MAX(views) OVER(PARTITION BY rang) AS max_views<br>
FROM pre_users)<br>
SELECT<br>
  id,<br>
  rang,<br>
  views<br>
FROM max_views_users<br>
WHERE views = max_views<br>
ORDER BY views DESC, id<br>
'''

### Задание 12
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.
<br>
'''<br>
WITH pre_table_2 AS(<br>
WITH pre_table AS(<br>
SELECT <br>
  CAST(u.creation_date AS date) AS creat_day,<br>
  COUNT(DISTINCT u.id) AS cnt<br>
FROM stackoverflow.users AS u<br>
WHERE u.creation_date BETWEEN '2008-11-01' AND '2008-12-01'<br>
GROUP BY creat_day)<br>
SELECT<br>
  creat_day,<br>
  EXTRACT(day FROM creat_day) AS day_creat,<br>
  cnt,<br>
  SUM(cnt) OVER(ORDER BY creat_day) AS total_users<br>
FROM pre_table)<br>
SELECT<br>
  day_creat,<br>
  cnt,<br>
  total_users<br>
FROM pre_table_2<br>
'''

### Задание 13
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

'''<br>
SELECT DISTINCT(u.id) AS uid,<br>
       MIN(p.creation_date) - u.creation_date<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.posts AS p ON p.user_id = u.id<br>
GROUP BY uid, u.creation_date<br>
'''

### Задание 14
Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.<br>
'''<br>
SELECT<br>
  CAST(DATE_TRUNC('month', creation_date) AS date) AS mon,<br>
  SUM(views_count) AS total_views<br>
FROM stackoverflow.posts<br>
GROUP BY mon<br>
ORDER BY total_views DESC;<br>
'''

### Задание 15
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.<br>
'''<br>
SELECT<br>
  (u.display_name),<br>
  COUNT(DISTINCT u.id) AS cnt<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.posts AS p ON p.user_id = u.id<br>
JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id<br>
WHERE pt.type = 'Answer'<br>
  AND (DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)<br>
  AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month')<br>
GROUP BY u.display_name<br>
HAVING COUNT(pt.id) > 100<br>
ORDER BY u.display_name<br>
'''

### Задание 16
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.<br>
'''<br>
WITH temp AS (<br>
SELECT<br>
  DISTINCT u.id AS uid<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.posts AS p ON p.user_id = u.id<br>
WHERE (u.creation_date BETWEEN '2008-09-01' AND '2008-10-01')<br>
  AND (p.creation_date BETWEEN '2008-12-01' AND '2009-01-01')<br>
GROUP BY uid<br>
HAVING COUNT(p.id) > 0)<br>

SELECT<br>
  CAST(DATE_TRUNC('month', p.creation_date) AS date) AS mont,<br>
  COUNT(p.id) AS cnt<br>
FROM stackoverflow.posts AS p<br>
JOIN temp ON temp.uid = p.user_id<br>
GROUP BY mont<br>
ORDER BY mont DESC<br>
'''

### Задание 17
Используя данные о постах, выведите несколько полей:

- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.<br>
'''<br>
SELECT<br>
  user_id,<br>
  creation_date,<br>
  views_count,<br>
  SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date)<br>
FROM stackoverflow.posts<br>
ORDER BY user_id, creation_date<br>
'''

### Задание 18
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.<br>
'''<br>
WITH temp AS (<br>
SELECT<br>
  DISTINCT u.id,<br>
  COUNT(p.id) AS cnt<br>
FROM stackoverflow.users AS u<br>
JOIN stackoverflow.posts AS p ON u.id = p.user_id<br>
WHERE p.creation_date BETWEEN '2008-12-01' AND '2008-12-08'<br>
GROUP BY u.id)<br>
SELECT ROUND(AVG(cnt) - 1)<br>
FROM temp<br>
'''
### Задание 19

На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:

- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.<br>
'''<br>
WITH temp AS (<br>
SELECT<br>
  EXTRACT(MONTH FROM CAST(p.creation_date AS date)) AS month_number,<br>
  COUNT(DISTINCT p.id) AS cnt<br>
FROM stackoverflow.posts AS p<br>
WHERE p.creation_date BETWEEN '2008-09-01' AND '2008-12-31'<br>
GROUP BY month_number)<br>

SELECT<br>
  month_number,<br>
  cnt,<br>
  ROUND((cnt - LAG(cnt) OVER()) * 100.0 / LAG(cnt) OVER(), 2) AS diff_percent<br>
FROM temp<br>
'''

### Задание 20
Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:

- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

'''<br>
WITH profile AS(<br>
SELECT<br>
  user_id,<br>
  COUNT(id) AS cnt<br>
FROM stackoverflow.posts<br>
GROUP BY user_id<br>
ORDER BY cnt DESC<br>
LIMIT 1),<br>

week_number AS (<br>
SELECT<br>
  creation_date,<br>
  EXTRACT(WEEK FROM creation_date) AS number_of_week<br>
FROM stackoverflow.posts AS p<br>
JOIN profile AS prof ON p.user_id = prof.user_id<br>
WHERE creation_date BETWEEN '2008-10-01' AND '2008-11-01')<br>

SELECT <br>
  DISTINCT number_of_week,<br>
  MAX(creation_date) OVER(PARTITION BY number_of_week)<br>
FROM week_number<br>
ORDER BY number_of_week;<br>
'''